In [506]:
# 패키지 로딩하기
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os, glob

In [668]:
# 하이퍼파라미터 설정하기
learning_rate   = 0.001
training_epochs = 30

In [508]:
# 데이터 불러오는 함수 만들기 : lms에서 인용함
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data = 300   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size       = 28
    color          = 3
    
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs   = np.zeros(number_of_data * img_size * img_size * color, dtype = np.int32).reshape(number_of_data, img_size, img_size,color)
    labels = np.zeros(number_of_data, dtype = np.int32)
    
    idx = 0
    for file in glob.iglob(img_path + '/scissor/*.jpg'):
        img             = np.array(Image.open(file), dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]     = 0      # 가위 : 0
        idx             = idx + 1

    for file in glob.iglob(img_path + '/rock/*.jpg'):
        img             = np.array(Image.open(file), dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]     = 1      # 바위 : 1
        idx             = idx + 1       
    
    for file in glob.iglob(img_path + '/paper/*.jpg'):
        img             = np.array(Image.open(file),dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]     = 2      # 보 : 2
        idx             = idx + 1
        
    print("데이터의 이미지 개수는",idx,"입니다.")
    return imgs, labels

In [509]:
# 훈련데이터 불러오기
(x_train1, y_train1) = load_data("aiffel/traning1_rock_scissor_paper")
(x_train2, y_train2) = load_data("aiffel/traning2_rock_scissor_paper")
(x_train3, y_train3) = load_data("aiffel/traning3_rock_scissor_paper")
(x_train4, y_train4) = load_data("aiffel/traning4_rock_scissor_paper")
(x_train5, y_train5) = load_data("aiffel/traning5_rock_scissor_paper")
(x_train6, y_train6) = load_data("aiffel/traning6_rock_scissor_paper")

데이터의 이미지 개수는 300 입니다.
데이터의 이미지 개수는 300 입니다.
데이터의 이미지 개수는 300 입니다.
데이터의 이미지 개수는 300 입니다.
데이터의 이미지 개수는 300 입니다.
데이터의 이미지 개수는 300 입니다.


In [510]:
# 훈련데이터 합치기
x_train = np.concatenate([x_train1, x_train2, x_train3, x_train4, x_train5, x_train6])
y_train = np.concatenate([y_train1, y_train2, y_train3, y_train4, y_train5, y_train6])
print(len(x_train), len(y_train))

1800 1800


In [511]:
# test데이터 불러오기
(x_test1, y_test1) = load_data("aiffel/test1_rock_scissor_paper")
(x_test2, y_test2) = load_data("aiffel/test2_rock_scissor_paper")
(x_test3, y_test3) = load_data("aiffel/test3_rock_scissor_paper")
(x_test4, y_test4) = load_data("aiffel/test4_rock_scissor_paper")

데이터의 이미지 개수는 300 입니다.
데이터의 이미지 개수는 300 입니다.
데이터의 이미지 개수는 300 입니다.
데이터의 이미지 개수는 300 입니다.


In [512]:
# 데이터 정규화 하기
x_train_normal  = x_train / 255.0  # 훈련 데이터
x_test1_normal  = x_test1 / 255.0  # test1 데이터
x_test2_normal  = x_test2 / 255.0  # test2 데이터
x_test3_normal  = x_test3 / 255.0  # test3 데이터
x_test4_normal  = x_test4 / 255.0  # test4 데이터

In [513]:
# 정규화 확인하기
print(np.min(x_train_normal), np.max(x_train_normal))   # 훈련 데이터
print(np.min(x_test1_normal), np.max(x_test1_normal))   # test1 데이터
print(np.min(x_test2_normal), np.max(x_test2_normal))   # test2 데이터
print(np.min(x_test3_normal), np.max(x_test3_normal))   # test3 데이터
print(np.min(x_test4_normal), np.max(x_test4_normal))   # test4 데이터

0.0 1.0
0.00784313725490196 1.0
0.0 1.0
0.00784313725490196 1.0
0.027450980392156862 0.9686274509803922


In [514]:
# 불러온 데이터 shape 확인하기
# (데이터 갯수, 가로, 세로, 깊이(색깔이 흑백이면 1, 컬러면 RGB를 뜻하는 3)

# 훈련 데이터
print(x_train_normal.shape)
print(y_train.shape)
# test1 데이터
print(x_test1_normal.shape)
print(y_test1.shape)
# test2 데이터
print(x_test2_normal.shape)
print(y_test2.shape)
# test3 데이터
print(x_test3_normal.shape)
print(y_test3.shape)
# test4 데이터
print(x_test4_normal.shape)
print(y_test4.shape)

(1800, 28, 28, 3)
(1800,)
(300, 28, 28, 3)
(300,)
(300, 28, 28, 3)
(300,)
(300, 28, 28, 3)
(300,)
(300, 28, 28, 3)
(300,)


In [669]:
# 모델 만들기

# 모델 생성
tf_model = tf.keras.Sequential()
# 계층 1
tf_model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3), input_shape = (28, 28, 3), activation = "relu"))
tf_model.add(tf.keras.layers.MaxPooling2D(pool_size = (1, 1)))
tf_model.add(tf.keras.layers.Dropout(0.3))
# 계층 2
tf_model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3), activation = "relu"))
tf_model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
tf_model.add(tf.keras.layers.Dropout(0.3))
# 계층 3
tf_model.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu"))
tf_model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
tf_model.add(tf.keras.layers.Dropout(0.3))
# 계층 4
tf_model.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu"))
tf_model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
tf_model.add(tf.keras.layers.Dropout(0.3))
# 계층 5
tf_model.add(tf.keras.layers.Flatten())
tf_model.add(tf.keras.layers.Dropout(0.3))
tf_model.add(tf.keras.layers.Dense(units = 512, activation = "relu"))
tf_model.add(tf.keras.layers.Dense(units = 3, activation = "softmax"))

# 모델 확인하기
tf_model.summary()

Model: "sequential_96"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_309 (Conv2D)          (None, 26, 26, 64)        1792      
_________________________________________________________________
max_pooling2d_297 (MaxPoolin (None, 26, 26, 64)        0         
_________________________________________________________________
dropout_53 (Dropout)         (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_310 (Conv2D)          (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_298 (MaxPoolin (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_311 (Conv2D)          (None, 10, 10, 128)     

In [670]:
# 모델 Loss, Optimizer, Metrics 설정하기
tf_model.compile(loss = "sparse_categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

In [671]:
# 모델 훈련하기
tf_model.fit(x_train_normal, y_train, epochs = training_epochs)

Epoch 1/30
57/57 [==============================] - 0s 4ms/step - loss: 1.1048 - accuracy: 0.3472
Epoch 2/30
57/57 [==============================] - 0s 3ms/step - loss: 1.0994 - accuracy: 0.3311
Epoch 3/30
57/57 [==============================] - 0s 3ms/step - loss: 1.0991 - accuracy: 0.3456
Epoch 4/30
57/57 [==============================] - 0s 3ms/step - loss: 1.0649 - accuracy: 0.4111
Epoch 5/30
57/57 [==============================] - 0s 3ms/step - loss: 0.9841 - accuracy: 0.4878
Epoch 6/30
57/57 [==============================] - 0s 3ms/step - loss: 0.8692 - accuracy: 0.5844
Epoch 7/30
57/57 [==============================] - 0s 3ms/step - loss: 0.7035 - accuracy: 0.6617
Epoch 8/30
57/57 [==============================] - 0s 3ms/step - loss: 0.5587 - accuracy: 0.7378
Epoch 9/30
57/57 [==============================] - 0s 3ms/step - loss: 0.5414 - accuracy: 0.7494
Epoch 10/30
57/57 [==============================] - 0s 3ms/step - loss: 0.4048 - accuracy: 0.8233
Epoch 11/30
57/57 [

In [672]:
# 모델 시험하기
evaluation1 = tf_model.evaluate(x_test1_normal, y_test1)
evaluation2 = tf_model.evaluate(x_test2_normal, y_test2)
evaluation3 = tf_model.evaluate(x_test3_normal, y_test3)
evaluation4 = tf_model.evaluate(x_test4_normal, y_test4)

10/10 [==============================] - 0s 2ms/step - loss: 1.8371 - accuracy: 0.6667
